# Testing WOFpy SOAP and REST for WOF/WaterML 1.1
Testing performed on cloud-based WOFpy endpoints with ODM2 timeseries MySQL and PostgreSQL backends. These resources are managed by WOFpy developers for WOFpy testing and validation.

Don Setiawan and Emilio Mayorga

Updates:
- 11/7/2017. Ran with the new EnviroDIY USU production endpoint. Made tweaks to make this run, including changes to site_and_variable_codes().
- 8/21/2017 Added the printing of Exception, and spaces between requests
- 8/19/2017. First general working version.

In [1]:
from urlparse import urljoin
import io

import requests
import ulmo
import sys
import traceback

/home/mayorga/miniconda/envs/odm2client/lib/python2.7/site-packages/ulmo/twc/kbdi/core.py:20: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  CSV_SWITCHOVER = pandas.tslib.Timestamp('2016-10-01')


In [2]:
print(ulmo.cuahsi.wof.__doc__)


    ulmo.cuahsi.wof
    ~~~~~~~~~~~~~~~

    `CUAHSI WaterOneFlow`_ web services

    .. _CUAHSI WaterOneFlow: http://his.cuahsi.org/wofws.html



In [3]:
print([obj for obj in dir(ulmo.cuahsi.wof) if not obj.startswith('__')])

['absolute_import', 'core', 'get_site_info', 'get_sites', 'get_values', 'get_variable_info']


In [4]:
def get_exception(exc_info):
    exc_type, exc_value, exc_traceback = exc_info
    print('*** {}:'.format(exc_type.__name__))
    traceback.print_exception(exc_type, exc_value, exc_traceback, file=sys.stdout)
    print('')

## Endpoints and testing resources

In [5]:
def site_and_variable_codes(networkcode):
    if 'postgres' in networkcode or 'wofpy' in networkcode:
        sitecode = '{netcode}:160065_Crosslands'.format(netcode=networkcode)
        variablecode = '{netcode}:EnviroDIY_Mayfly_Temp'.format(netcode=networkcode)
    elif 'mysql' in networkcode:
        sitecode = '{netcode}:USU-LBR-Mendon'.format(netcode=networkcode)
        variablecode = '{netcode}:USU36'.format(netcode=networkcode)
    
    return sitecode, variablecode

In [6]:
# server_base_url = 'http://54.186.36.247:8080'
# networkcodes = ['postgresqlodm2timeseries', 'mysqlodm2timeseries']
# networkcodes = ['postgresqlodm2timeseries']

server_base_url = 'http://data.envirodiy.org/'
networkcodes = ['wofpy']

## SOAP 1.1 Testing

In [7]:
wofpy_wsdl_url = '/soap/cuahsi_1_1/.wsdl'
for networkcode in networkcodes:
    url = urljoin(server_base_url, networkcode+wofpy_wsdl_url)
    sitecode, variablecode = site_and_variable_codes(networkcode)

    print('--------------------------')
    print('Testing {}'.format(url))
    print('--------------------------')
        
    print('SITES-----------------')
    try:
        sites = ulmo.cuahsi.wof.get_sites(url)
        
        print(sites.keys())
        print('')
    except:
        print('GetSites failed!')
        get_exception(sys.exc_info())

    
    print('SITE INFO {}-------------'.format(sitecode))
    try:
        siteinfo = ulmo.cuahsi.wof.get_site_info(url, site_code=sitecode)
        
        print(siteinfo['name'])
        print('')
    except:
        print('GetSiteInfo failed!')
        get_exception(sys.exc_info())
    
    print('VARIABLES-------------')
    try:
        variables = ulmo.cuahsi.wof.get_variable_info(url)
        
        print(variables.keys())
        print('')
    except:
        print('GetVariables failed!')
        get_exception(sys.exc_info())

    print('VARIABLE INFO {}-------------'.format(variablecode))
    try:
        variable = ulmo.cuahsi.wof.get_variable_info(url, variable_code=variablecode)
        
        print(variable['name'])
        print('')
    except:
        print('GetVariableInfo failed!')
        get_exception(sys.exc_info())
    
        
    print('VALUES----------------')    
    try:
        values = ulmo.cuahsi.wof.get_values(url, site_code=sitecode, variable_code=variablecode)
        
        print(len(values['values']))
        print('')
    except:
        print('GetValues failed!')
        get_exception(sys.exc_info())

--------------------------
Testing http://data.envirodiy.org/wofpy/soap/cuahsi_1_1/.wsdl
--------------------------
SITES-----------------
['wofpy:NHMU9S', 'wofpy:NHMU10S', 'wofpy:ZYU1', 'wofpy:PKCV2S', "wofpy:Craig's_Desk_Test", 'wofpy:office_test01', 'wofpy:LaSalleE', 'wofpy:RockyLow', 'wofpy:TKessler2', 'wofpy:Casia', 'wofpy:LaSalleN', 'wofpy:NHPK7S', 'wofpy:SHPK4S', 'wofpy:asdfsdf', 'wofpy:CRO-IT Closet', 'wofpy:USU_Mayfly_Test', 'wofpy:USU_STAGE_TEST', 'wofpy:Ben_Desk', 'wofpy:desk_sensors', 'wofpy:KINNI_Logger3', 'wofpy:USU-Logan', 'wofpy:NHPK8S', 'wofpy:PUPP2S', 'wofpy:asdfasdf', 'wofpy:Hurricane', 'wofpy:UofU_Test', 'wofpy:TESTzzz', 'wofpy:tate_test01', 'wofpy:PKCV3S', 'wofpy:JRains2', 'wofpy:JRains1', 'wofpy:Anthonys_Desk', 'wofpy:01', 'wofpy:Beth_office', 'wofpy:CRO_Office', 'wofpy:qwerqwer', 'wofpy:GermantownFS', 'wofpy:BeaverLow', 'wofpy:CME_415', 'wofpy:USU_Test_Site', 'wofpy:USU-Logan-Temp', 'wofpy:USU_STAGING_TEST', 'wofpy:RockyUp', 'wofpy:PUPP3S', 'wofpy:160065_Crosslan

## REST 1.1 Testing

In [8]:
wofpy_rest_url = '/rest/1_1/'
for networkcode in networkcodes:
    url = urljoin(server_base_url, networkcode+wofpy_rest_url)
    sitecode, variablecode = site_and_variable_codes(networkcode)

    print('--------------------------')
    print('Testing {}'.format(url))
    print('--------------------------')
    
    print('SITES-----------------')
    try:
        req = requests.get(urljoin(url, 'GetSites'))
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        sites = ulmo.waterml.v1_1.parse_site_infos(response_buffer)
        
        print(sites.keys())
        print('')
    except:
        print('GetSites failed!')
        get_exception(sys.exc_info())
    
    print('SITE INFO {}-------------'.format(sitecode))
    try:
        req = requests.get(urljoin(url, 'GetSiteInfo'), params={'site':sitecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        siteinfo = ulmo.waterml.v1_1.parse_sites(response_buffer)
        
        print(siteinfo[sitecode.split(':')[1]]['name'])
        print('')
    except:
        print('GetSiteInfo failed!')
        get_exception(sys.exc_info())
    
    print('VARIABLES-------------')
    try:
        req = requests.get(urljoin(url, 'GetVariables'))
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        variables = ulmo.waterml.v1_1.parse_variables(response_buffer)
        
        print(variables.keys())
        print('')
    except:
        print('GetVariables failed!')
        get_exception(sys.exc_info())

    print('VARIABLE INFO {}-------------'.format(variablecode))
    try:
        req = requests.get(urljoin(url, 'GetVariableInfo'), params={'variable':variablecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        variable = ulmo.waterml.v1_1.parse_variables(response_buffer)
        
        print(variable[variablecode.split(':')[1]]['name'])
        print('')
    except:
        print('GetVariableInfo failed!')
        get_exception(sys.exc_info())
    
        
    print('VALUES----------------')    
    try:
        req = requests.get(urljoin(url, 'GetValues'), params={'location':sitecode, 
                                                              'variable':variablecode})
        response_buffer = io.BytesIO(ulmo.util.to_bytes(req.content))
        values = ulmo.waterml.v1_1.parse_site_values(response_buffer)
        
        print(len(values[variablecode.split(':')[1]]['values']))
        print('')
    except:
        print('GetValues failed!')
        get_exception(sys.exc_info())

--------------------------
Testing http://data.envirodiy.org/wofpy/rest/1_1/
--------------------------
SITES-----------------
['KINNI_Logger1', 'PKCV2S', 'ZYU1', 'RockyUp', 'PUPP2S', 'Hurricane', 'TESTzzz', 'NHMU10S', 'USU_Logan', 'SHPK4S', 'asdfsdf', 'NHMU9S', 'CRO-IT Closet', 'Logan_Temp', 'qwerqwer', '01', 'Beth_office', 'UofU_Test', 'USU_Test_Site', 'NHPK7S', 'Ramsey', 'CME_415', 'CRO_Office', 'Casia', 'BeaverLow', 'USU-Logan-Temp', 'Ben_Desk', 'JRains2', 'tate_test01', 'KINNI_Logger3', 'KINNI_Logger2', 'KINNI_Logger5', 'KINNI_Logger4', 'KINNI_Logger6', 'PUPP3S', 'SHPK3S', 'desk_sensors', 'PKCV3S', 'JRains1', "Craig's_Desk_Test", 'TKessler2', 'NHPK8S', 'RockyLow', 'USU_STAGE_TEST', 'USU_STAGING_TEST', 'LaSalleE', 'USU-Logan', 'LaSalleN', 'office_test01', '160065_Crosslands', 'Anthonys_Desk', 'asdfasdf', 'USU_Mayfly_Test', 'GermantownFS']

SITE INFO wofpy:160065_Crosslands-------------
160065 CrosslandsPond - 10 variables

VARIABLES-------------
['Decagon_CTD-10_Temp', 'Apogee_SQ-2